In [1]:
# Import the modules
import numpy as np
import pandas as pd
from pathlib import Path
from sklearn.metrics import accuracy_score, balanced_accuracy_score, confusion_matrix, classification_report

---

## Split the Data into Training and Testing Sets

### Step 1: Read the `lending_data.csv` data from the `Resources` folder into a Pandas DataFrame.

In [2]:
# Read the CSV file from the Resources folder into a Pandas DataFrame
file_path = "Resources/lending_data.csv"
lending_data_df = pd.read_csv(file_path)

# Review the DataFrame
lending_data_df.head()

,loan_size,interest_rate,borrower_income,debt_to_income,num_of_accounts,derogatory_marks,total_debt,loan_status
0,10700.0,7.672,52800,0.431818,5,1,22800,0
1,8400.0,6.692,43600,0.311927,3,0,13600,0
2,9000.0,6.963,46100,0.349241,3,0,16100,0
3,10700.0,7.664,52700,0.430740,5,1,22700,0
4,10800.0,7.698,53000,0.433962,5,1,23000,0


### Step 2: Create the labels set (`y`)  from the “loan_status” column, and then create the features (`X`) DataFrame from the remaining columns.

In [3]:
# Separate the data into labels and features

# Separate the y variable, the labels
y = lending_data_df["loan_status"]

# Separate the X variable, the features
X = lending_data_df.drop(columns = ["loan_status"])

In [4]:
# Review the y variable Series
y[:5]

0    0
1    0
2    0
3    0
4    0
Name: loan_status, dtype: int64

In [5]:
# Review the X variable DataFrame
X[:5]

,loan_size,interest_rate,borrower_income,debt_to_income,num_of_accounts,derogatory_marks,total_debt
0,10700.0,7.672,52800,0.431818,5,1,22800
1,8400.0,6.692,43600,0.311927,3,0,13600
2,9000.0,6.963,46100,0.349241,3,0,16100
3,10700.0,7.664,52700,0.430740,5,1,22700
4,10800.0,7.698,53000,0.433962,5,1,23000


### Step 3: Check the balance of the labels variable (`y`) by using the `value_counts` function.

In [6]:
# Check the balance of our target values
y.value_counts()

0    75036
1     2500
Name: loan_status, dtype: int64

The above cell tells us that roughly 75,000 loans are healthy and 2,500 loans have a high risk of defaulting

### Step 4: Split the data into training and testing datasets by using `train_test_split`.

In [7]:
# Import the train_test_learn module
from sklearn.model_selection import train_test_split

# Split the data using train_test_split
# Assign a random_state of 1 to the function
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1, stratify=y)

---

## Create a Logistic Regression Model with the Original Data

###  Step 1: Fit a logistic regression model by using the training data (`X_train` and `y_train`).

In [8]:
# Import the LogisticRegression module from SKLearn
from sklearn.linear_model import LogisticRegression

# Instantiate the Logistic Regression model
# Assign a random_state parameter of 1 to the model
classifier = LogisticRegression(solver='lbfgs', random_state=1)

# Fit the model using training data
classifier.fit(X_train, y_train)

LogisticRegression(random_state=1)

### Step 2: Save the predictions on the testing data labels by using the testing feature data (`X_test`) and the fitted model.

In [9]:
# Make a prediction using the testing data
predictions = classifier.predict(X_test)
results = pd.DataFrame({"Prediction": predictions, "Actual": y_test})
results.head()

,Prediction,Actual
36831,0,0
75818,0,1
36563,0,0
13237,0,0
43292,0,0


### Step 3: Evaluate the model’s performance by doing the following:

* Calculate the accuracy score of the model.

* Generate a confusion matrix.

* Print the classification report.

In [10]:
# Print the balanced_accuracy score of the model
balanced_accuracy = balanced_accuracy_score(y_test, predictions)
print(f"The balanced accuracy score for this model is {round(balanced_accuracy*100, 2)}%")

The balanced accuracy score for this model is 94.43%


In this case, we use a balanced accuracy score because we have such an unbalanced dataset. The testing model contains 18,759 healthy loans and 625 high-risk loans, which is orders of magnitude fewer. If the model just predicted every loan was healthy, it would get an accuracy score of almost 97%. However, its the high-risk loans that are most important to us to separated out from the healthy loans. The balanced_accuracy_score takes this into account, and gives us a score of 94% accuracy, which is still quite good.

In [11]:
# Generate a confusion matrix for the model
cm = confusion_matrix(y_test, predictions)

cm_df = pd.DataFrame(
    cm, index=["Actual Healthy", "Actual High-Risk"], columns=["Predicted Healthy", "Predicted High-Risk"]
)

cm_df

,Predicted Healthy,Predicted High-Risk
Actual Healthy,18679,80
Actual High-Risk,67,558


In [12]:
# Print the classification report for the model
classification_report_output = classification_report(y_test, predictions)
print(classification_report_output)

              precision    recall  f1-score   support

           0       1.00      1.00      1.00     18759
           1       0.87      0.89      0.88       625

    accuracy                           0.99     19384
   macro avg       0.94      0.94      0.94     19384
weighted avg       0.99      0.99      0.99     19384



### How well does the logistic regression model predict both the `0` (healthy loan) and `1` (high-risk loan) labels?

In general, the logistic regression model does a good job of predicting healthy loans. Of the total 18759 actually healthy loans, 18679 were correctly predicted as healthy, which is a 99.57% accuracy rate. This is also reflected in the 100% precision, and 100% recall for healthy loans.

However, the model doesn't do as well for predicting high-risk loans. Of the total 625 loans, 67 were incorrectly predicted as healthy, which is a little over 10%. This is reflected in the 87% precision (which is the ratio of correct high-risk loans to all predicted high-risk loans, even those that were actually healthy) and the 89% recall (which ths ratio of correctly identified high-risk loans over everything that should have been considered high-risk loans, even those that were incorrectly identified as healthy).

As discussed above, this imbalance in the precision and recall for the healthy loans as compared to the high-risk loans may be attributed to the unbalanced nature of the dataset, with the sample size of healthy loans being significantly larger than the sample size of high-risk loans. 

The next steps of this process will use a RandomOverSampler to try to correct for this imbalance in the dataset.

---

## Predict a Logistic Regression Model with Resampled Training Data

### Step 1: Use the `RandomOverSampler` module from the imbalanced-learn library to resample the data. Be sure to confirm that the labels have an equal number of data points. 

In [13]:
!pip install imblearn

In [14]:
# Import the RandomOverSampler module form imbalanced-learn
from imblearn.over_sampling import RandomOverSampler

# Instantiate the random oversampler model
# # Assign a random_state parameter of 1 to the model
ros = RandomOverSampler(random_state = 1)

# Fit the original training data to the random_oversampler model
X_over, y_over = ros.fit_resample(X_train, y_train)

In [15]:
# Count the distinct values of the resampled labels data
y_over.value_counts()

1    56277
0    56277
Name: loan_status, dtype: int64

The above cell tells us that there are now 56277 loans in both the healthy and high-risk categories.  This is because the RandomOverSampler library has randomly duplicated examples from the high-risk category in order to make the original dataset less imbalanced.

(Information taken from [MachineLearningMastery](https://machinelearningmastery.com/random-oversampling-and-undersampling-for-imbalanced-classification/) )

### Step 2: Use the `LogisticRegression` classifier and the resampled data to fit the model and make predictions.

In [16]:
# Instantiate the Logistic Regression model
# Assign a random_state parameter of 1 to the model
classifier_ros = LogisticRegression(solver='lbfgs', random_state=1)

# Fit the model using the resampled training data
classifier_ros.fit(X_over, y_over)

# Make a prediction using the testing data
predictions_ros = classifier_ros.predict(X_test)
results = pd.DataFrame({"Prediction": predictions_ros, "Actual": y_test})
results.head()

,Prediction,Actual
36831,0,0
75818,1,1
36563,0,0
13237,0,0
43292,0,0


### Step 3: Evaluate the model’s performance by doing the following:

* Calculate the accuracy score of the model.

* Generate a confusion matrix.

* Print the classification report.

In [17]:
# Print the balanced_accuracy score of the model 
balanced_accuracy = balanced_accuracy_score(y_test, predictions_ros)

accuracy = accuracy_score(y_test, predictions_ros)

print(f"The balanced accuracy score for this model is {round(balanced_accuracy*100, 2)}%")
print(f"The accuracy score for this model is {round(accuracy*100, 2)}%")

The balanced accuracy score for this model is 99.6%
The accuracy score for this model is 99.52%


In [18]:
# Generate a confusion matrix for the model
cm_ros = confusion_matrix(y_test, predictions_ros)

cm_ros_df = pd.DataFrame(
    cm_ros, index=["Actual Healthy", "Actual High-Risk"], columns=["Predicted Healthy", "Predicted High-Risk"]
)

cm_ros_df

,Predicted Healthy,Predicted High-Risk
Actual Healthy,18668,91
Actual High-Risk,2,623


In [19]:
# Print the classification report for the model
classification_report_ros = classification_report(y_test, predictions_ros)
print(classification_report_ros)

              precision    recall  f1-score   support

           0       1.00      1.00      1.00     18759
           1       0.87      1.00      0.93       625

    accuracy                           1.00     19384
   macro avg       0.94      1.00      0.96     19384
weighted avg       1.00      1.00      1.00     19384



### How well does the logistic regression model, fit with oversamled data, predict both the `0` (healthy loan) and `1` (high-risk loan) labels?

In this case, the logistic regresseion model using oversampled data does a better job than using the raw, unskewed data. Of the total 18759 total healthy loans, 18668 were correctly identified as healthy, which is a 99.51% accuracy rate, which is approximately the same as the 99.57% accuracy rate using the raw data. Again, the precision and recall were both 100% for healthy loans.

The issue with using raw data, however, was that the accuracy of predicting the high-risk loans was lower, and around 10% of the loans were incorrectly identified as healthy. Using the random over-sampling method corrected this issue. Out of the 625 high-risk loans, only 2 were misidentified as healthy, leading to a 99.7% accuracy rate.

The recall for high-risk loans was 100%, in line with the recall for healthy loans. However, the precision for high-risk loans is still only 87%, which was the case before the oversampling. This indicates we still have a loans that were healthy that were incorrectly identified as high-risk.

This is definitely an improvement, but it does mean that if this model is to be used, we should pay particularly close attention to high-risk loans. According to this data, there is very little chance that an actually high-risk loan is going to be misidentified as healthy, but there is a significant chance that a loan identified as high-risk will actually be healthy. With this in mind, taking a second look over these applications is warranted, to avoid deterring customers that are likely to pay back the balance of their loan.